In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scDeBussy
from scDeBussy.pl import *
import seaborn as sns
from matplotlib.colors import LogNorm
import pandas as pd
import matplotlib.pyplot as plt
import PyComplexHeatmap as pch
from sklearn.neighbors import KernelDensity

In [ ]:
summary_df = pd.read_csv(summary_df_path, index_col=0)
gene_curves = pd.read_csv(aggregated_curves_path, index_col=0)
scores_df = pd.read_csv(scores_df_path, index_col=0)
hvg_df = pd.read_csv(hvg_df_path, sep="\t")
hvg_df.columns = ["gene", "rank", "batches"]
clusters = cluster_ordering.split("_")
cluster_cutoff = [i/len(clusters) for i in range(len(clusters) - 1)]
color_map = {'NSCLC': 'gold',
             'SCLC-A': 'tab:red',
             'SCLC-N': 'tab:cyan'}
colors = [color_map[x] for x in clusters]

In [ ]:
scores_df.sort_values(['GCV'], ascending=[True]).merge(hvg_df, on='gene').head(n=50)

In [ ]:

sorted_gene_curve, row_colors, col_colors, categories = scDeBussy.process_gene_data(scores_df, gene_curves, colors, cluster_cutoff,
                                                                                       n_clusters=3, n_init=1, MI_threshold=0.5,
                                                                                       GCV_threshold=0.98, 
                                                                                       label_names=['Early', 'Middle', 'Late'])
print(f"{sorted_gene_curve.shape[0]} genes left after filtering.")
plot_kshape_clustering(sorted_gene_curve, categories, ['Early', 'Middle', 'Late'], alpha=0.01)

In [ ]:
gene_info = pd.DataFrame({'gene': sorted_gene_curve.index, 'category': categories})
gene_sets = ['CellMarker_2024']
results = pd.DataFrame()  # Initialize an empty DataFrame to store results

for category in gene_info.category.unique():
    gene_list = gene_info.gene[gene_info.category == category]
    results_category = scDeBussy.enrichr(gene_list, gene_sets)
    results_category.loc[:,'category'] = category
    results = pd.concat([results, results_category])
    
results.loc[:,'overlap'] = results.Overlap.apply(lambda x: int(x.split('/')[0]))
results = results[(results['Adjusted P-value'] < 0.05) & (results['overlap'] > 10)]
results = results.drop_duplicates(subset="Genes")
pd.set_option('display.max_rows', None)
gene_sets = results.sort_values(by=["category", "Odds Ratio"], ascending = [True, False]).reset_index(drop=True)
gene_sets.to_csv(f"{cluster_ordering}_enrichR.csv")

In [ ]:
gene_sets.groupby('category').apply(lambda x: x.sort_values(by='Combined Score', ascending=False).head(5))

In [ ]:
genes = gene_sets['Genes'].str.split(';').explode().str.strip().unique()
gene_to_terms = {}
for index, row in gene_sets.iterrows():
    term = row['Term']
    genes = row['Genes'].split(';')
    combined_score = row['Combined Score']
    for gene in genes:
        if gene not in gene_to_terms:
            gene_to_terms[gene] = {'term': term}
        else:
            gene_to_terms[gene]['term'] = gene_to_terms[gene]['term'] + ';' + term
final_gene_to_term = {gene: info['term'] for gene, info in gene_to_terms.items()}
sorted_gene_curve_with_annot = sorted_gene_curve.copy()
sorted_gene_curve_with_annot.loc[:,'gene_sets'] = sorted_gene_curve_with_annot.index.map(final_gene_to_term)
sorted_gene_curve_with_annot.loc[:,'cluster'] = row_colors.values

cell_type_colors = {
    'NSCLC': 'gold', 'SCLC-A': 'tab:red', 'SCLC-N': 'tab:cyan'
}
cell_types = col_colors.map(lambda x: 'NSCLC' if x == 0 else 'SCLC-A' if x == 1 else 'SCLC-N')
clusters = row_colors
cluster_colors = {}
for i, category in enumerate(gene_info['category'].unique()):
    cluster_colors[category] = plt.get_cmap('viridis')(i / len(gene_info['category'].unique()))

df_right = sorted_gene_curve_with_annot['gene_sets']
df_right.index = sorted_gene_curve.index
df_left = sorted_gene_curve_with_annot['cluster']
df_left.index = sorted_gene_curve.index

In [ ]:
density = None
left_annotation_columns = None

In [ ]:
plot_kde_heatmap(cluster_colors, cell_types, cell_type_colors, sorted_gene_curve, df_left, density, left_annotation_columns)